## 7 畳み込みニューラルネットワーク（CNN）

## 7.2 im2colとcol2im

### 7.2.3 im2colの実装 -シンプルなim2col-

In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w): #1
    
    img_h, img_w = image.shape # 入力画像の高さ、幅
    
    # 生成される行列のサイズ
    cols = np.zeros((flt_h * flt_w, out_h * out_w))
    
    for h in range(out_h): 
        h_lim = h + flt_h
        for w in range(out_w):
            w_lim = w + flt_w
            cols[:, h * out_w + w] = image[h:h_lim, w:w_lim].reshape(-1)
            
    return cols

In [3]:
img = np.array([[1, 2, 3, 4], 
                [5, 6, 7, 8], 
                [9, 10, 11, 12], 
                [13, 14, 15, 16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [4]:
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w):
    
    img_h, img_w = image.shape # 入力画像の高さ、幅
    
    # 生成される行列のサイズ
    cols = np.zeros((flt_h, flt_w, out_h, out_w))
    
    for h in range(flt_h): 
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[h, w, :, :] = image[h:h_lim, w:w_lim]
    
    cols = cols.reshape(flt_h*flt_w, out_h*out_w)
            
    return cols

In [5]:
img = np.array([[1, 2, 3, 4], 
                [5, 6, 7, 8], 
                [9, 10, 11, 12], 
                [13, 14, 15, 16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


### 7.2.4 im2colの実装 -im2colの実用化

In [6]:
# バッチとチャンネルに対応
def im2col(images, flt_h, flt_w, out_h, out_w):
   
    n_bt, n_ch, img_h, img_w = images.shape # バッチサイズ、チャンネル数、入力画像高さ、幅
    
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

    for h in range(flt_h):
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[:, :, h, w, :, :] = images[:, :, h:h_lim, w:w_lim]

    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w, n_bt*out_h*out_w)
    return cols

In [7]:
img = np.array([[[[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10,11,12],
                  [13,14,15,16]]]])
cols = im2col(img, 2, 2, 3, 3)


バッチとチャンネルに対応しパディングとストライドを考慮したim2col

In [8]:
# coding: UTF-8
#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
def im2col(images, flt_h, flt_w, out_h, out_w, stride, pad):
   
    n_bt, n_ch, img_h, img_w = images.shape
    
    img_pad = np.pad(images, [(0,0), (0,0), (pad, pad), (pad, pad)], "constant")
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))

    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            cols[:, :, h, w, :, :] = img_pad[:, :, h:h_lim:stride, w:w_lim:stride]

    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w, n_bt*out_h*out_w)
    return cols

#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
#1 image:入力画像，
#  flt_h:フィルタの高さ，flt_w:フィルタの幅, out_h:出力画像の高さ, out_w:出力画像の幅

In [9]:
img = np.array([[[[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10,11,12],
                  [13,14,15,16]]]])
cols = im2col(img, 2, 2, 3, 3, 1, 0)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


### 7.2.6 col2im の実装

In [10]:
# coding: UTF-8
#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
def col2im(cols, img_shape, flt_h, flt_w, out_h, out_w, stride, pad):
 
    n_bt, n_ch, img_h, img_w = img_shape
    
    # 6次元配列に分解して軸を入れ替えて，配列形状は (B, C, Fh, Fw, Oh, Ow) となる．
    cols = cols.reshape(n_ch, flt_h, flt_w, n_bt, 
                        out_h, out_w).transpose(3, 0, 1, 2, 4, 5) 
    
    # 変換後の画像を格納する4次元配列の作成
    images = np.zeros((n_bt, n_ch, img_h + 2*pad + stride - 1,
                       img_w + 2 * pad + stride -1)) 
    # stride-1を加えるのは画像がストライドの値で割り切れない場合の対策
    
    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            images[:, :, h:h_lim:stride, w:w_lim:stride] += cols[:, :, h, w, :, :]
            # imagesにcolsにおけるフィルタの該当箇所を格納する

    return images[:, :, pad:img_h+pad, pad:img_w+pad]
            # パディングの分をスライスによって取り除いている

#---+--10|----+--20|----+--30|----+--40|----+--50|----+--60|----+--70|----+--80|
#1 cols:引数として受け取る行列
#  受け取るcolsの形状は (n_ch * flt_h * flt_w, n_bt * out_h * ouy_w)
#  n_ch:チャンネル数，n_bt:バッチサイズ数,

In [11]:
cols = np.ones((4, 4))
img_shape = (1, 1, 3, 3)
images = col2im(cols, img_shape, 2, 2, 2, 2, 1, 0)
print(images)

[[[[1. 2. 1.]
   [2. 4. 2.]
   [1. 2. 1.]]]]


## 7.3 畳み込み層の実装

### 7.3.1 実装の概要

In [12]:
class ConvLayer: 
    
    # x_ch: 入力チャンネル数, x_h: 入力画像高さ, x_w: 入力画像幅
    # n_fit: フィルタ数, flt_h: フィルタ高さ, flt_w: フィルタ幅
    # stride: ストライド幅, pad: パディング幅
    
    def __init__(self, x_ch, x_h, x_w, n_flt, flt_h, flt_w, 
                 stride, pad):
        
        # パラメータをまとめる
        self.params = (x_ch, x_h, x_w, n_flt, flt_h, flt_w, 
                       stride, pad)
        
        # フィルタとバイアスの初期値
        self.w = wb_width * np.random.randn(n_flt, x_ch, flt_h, flt_w)
        self.b = wb_width * np.random.randn(1, n_flt)
        
        # 出力画像のサイズ
        self.y_ch = n_flt # 出力チャンネル数
        self.y_n = (x_h - flt_h + 2*pad) // stride + 1 # 出力高さ, //: 切り捨て除算
        self.y_w = (x_w - flt_w + 2*pad) // stride + 1 # 出力幅

### 7.3.2 順伝播

In [13]:
    def forward(self, x): 
        n_bt = x.shape[0]
        x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
        y_ch, y_h, y_w = self.y_ch, self.y_ch, self.y_w
        
        # 入力画像とフィルタを行列に変換
        self.cols = im2col(x, flt_h, flt_w, y_h, y_w, stride, pad)
        self.w_col = self.w.reshape(n_flt, x_ch*flt_h*flt_w)
        
        # 出力の計算
        u = np.dot(self.w_col, self.cols).T + self.b
        self.u = u.reshape(n_bt, y_h, y_w, y_ch).transpose(0, 3, 1, 2)
        self.y = np.where(self.u <= 0, 0, self.u)

### 7.3.3 逆伝播

In [14]:
    def backward(self, grad_y): 
        n_bt = grad_y.shape[0]
        x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad =self.params
        y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
        
        # delta
        delta = grad_y * np.where(self.u <= 0, 0, 1)
        delta = delta.transpose(0, 2, 3, 1).reshape(n_bt*y_h*y_w, y_ch)
        
        # フィルタとバイアスの勾配
        grad_w = np.dot(self.cols, delta)
        self.grad_w = grad_w.T.reshape(n_flt, x_ch, flt_h, flt_w)
        self.grad_b = np.sum(delta, axis=0)
        
        # 入力の勾配
        grad_cols = np.dot(delta, self.w_col)
        x_shape = (n_bt, x_ch, x_h, x_w)
        self.grad_x = col2im(grad_cols.T, x_shape, flt_h, 
                            flt_w, y_h, y_w, stride, pad)

## 7.4 プーリング層の実装

### 7.4.1 プーリング層の概要

In [15]:
class PoolingLayer: 
    
    # x_ch: 入力チャンネル数, x_h: 入力画像高さ, x_w: 入力画像幅
    # ppol: プーリング領域のサイズ, pad: パディング幅
    
    def __init__(self, x_ch, x_h, x_w, pool, pad): 
        
        # パラメータをまとめる
        self.params = (x_ch, x_h, x_w, pool, pad)
        
        self.y_ch = x_ch # 出力チャンネル数
        self.y_h = x_h // pool if x_h % pool == 0 else x_h // pool + 1 # 出力高さ
        self.y_w = x_w // pool if x_w % pool == 0 else x_w // pool + 1 # 出力幅
        

### 7.4.2 順伝播

In [16]:
def forward (self, x): 
    n_bt = x.shape[0]
    x_ch, x_h, x_w, pool, pad = self.params
    y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
    
    # 入力画像を行列に変換
    cols = im2col(x, pool, pool, y_h, pool, pad)
    cols = cols.T.reshape(n_bt * y_h * y_w * x_ch, pool * pool)
    
    # 出力の計算: Maxプーリング
    y = np.max(cols, axis = 1)
    self.y = y.reshape(n_bt, y_h, y_w, x_ch).transpose(0, 3, 1, 2)
    
    # 最大値のインデックスを保存
    self.max_index = np.argmax(cols, axis = 1)

### 7.4.3 逆伝播

In [17]:
def backward(self, grad_y): 
    n_bt = grad_y.shape[0]
    x_ch, x_h, x_w, pool, pad = self.params
    y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
    
    # 出力の勾配の値を入れ替え
    grad_y = grad_y.transpose(0, 2, 3, 1)
    
    # 行列を作成
    grad_cols = np.zeros((pool * pool, grad_y.size))
    
    # 各列の最大値であった要素にのみ出力の勾配を入れる
    grad_cols[self.max_index.reshape(-1), 
              np.arange(grad_y.size)] = grad_y.reshape(-1)
    grad_cols = grad_cols.reshape(pool, pool, n_bt, y_h, y_w, y_ch)
    grad_cols = grad_cols.transpose(5, 0, 1, 2, 3, 4)
    grad_cols = grad_cols.reshape(y_ch * pool * pool, n_bt * y_h * y_w)
    
    # 入力の勾配
    x_shape = (n_bt, x_ch, x_h, x_w)
    self.grad_x = col2im(grad_cols, x_shape, pool, pool, y_h, y_w, pool, pad)